# Data Processing : Spark SQL, Dataframes, Datasets 
suite (3) ...

## Preprocessing de Airlines Data

In [1]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    appName("Basic Transformations").
    getOrCreate

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.43.47:4044
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1625091472724)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@25c9ae87


In [2]:
val airlines_path = "datasets/airlines_all.05p.csv"

airlines_path: String = datasets/airlines_all.05p.csv


In [3]:
!head "datasets/airlines_all.05p.csv"

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
1988,1,9,6,1348,1331,1458,1435,PI,942,NA,70,64,NA,23,17,SYR,BWI,273,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,YES
1988,1,29,5,1339,1331,1442,1435,PI,942,NA,63,64,NA,7,8,SYR,BWI,273,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,YES
1988,1,23,6,950,950,1041,1050,PI,943,NA,51,60,NA,-9,0,LGA,SYR,198,NA,NA,0,NA,0,NA,NA,NA,NA,NA,NO,NO
1988,1,18,1,1124,1110,1213,1145,PI,943,NA,49,35,NA,28,14,SYR,BUF,134,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,YES
1988,1,10,7,1503,1500,1602,1550,PI,944,NA,59,50,NA,12,3,JFK,UCA,191,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,YES
1988,1,30,6,1500,1500,1558,1550,PI,944,NA,58,50,NA,8,0,JFK,UCA,191,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,NO
1988,1,20,3,1750,1705,1900,1810,PI,944,NA,70,65,NA,50,45

In [4]:
val airlines_all = spark.
    read.
    option("header","true").
    csv(airlines_path)

airlines_all: org.apache.spark.sql.DataFrame = [Year: string, Month: string ... 29 more fields]


In [5]:
airlines_all.printSchema

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDelay:

In [6]:
val airlines = airlines_all.
    select("Year", "Month", "DayOfMonth",
           "DepDelay", "ArrDelay", "UniqueCarrier", 
           "FlightNum", "IsArrDelayed", "IsDepDelayed"
          )

airlines: org.apache.spark.sql.DataFrame = [Year: string, Month: string ... 7 more fields]


In [19]:
airlines.show

+----+-----+----------+--------+--------+-------------+---------+------------+------------+
|Year|Month|DayOfMonth|DepDelay|ArrDelay|UniqueCarrier|FlightNum|IsArrDelayed|IsDepDelayed|
+----+-----+----------+--------+--------+-------------+---------+------------+------------+
|1988|    1|         9|      17|      23|           PI|      942|         YES|         YES|
|1988|    1|        29|       8|       7|           PI|      942|         YES|         YES|
|1988|    1|        23|       0|      -9|           PI|      943|          NO|          NO|
|1988|    1|        18|      14|      28|           PI|      943|         YES|         YES|
|1988|    1|        10|       3|      12|           PI|      944|         YES|         YES|
|1988|    1|        30|       0|       8|           PI|      944|         YES|          NO|
|1988|    1|        20|      45|      50|           PI|      944|         YES|         YES|
|1988|    1|        10|       6|       2|           PI|      944|         YES|  

Obtenir le décompte des vols qui partent en retard à l'origine et arrivent à destination tôt ou à l'heure.

In [7]:
airlines.
    filter("IsDepDelayed = 'YES' AND IsArrDelayed = 'NO'").
    count

res1: Long = 44985


In [8]:
import org.apache.spark.sql.functions.col

import org.apache.spark.sql.functions.col


In [9]:
airlines.
    filter(col("IsDepDelayed") === "YES" and col("IsArrDelayed") === "NO").
    count

res2: Long = 44985


In [10]:
import spark.implicits._

import spark.implicits._


In [11]:
airlines.
    filter($"IsDepDelayed" === "YES" and $"IsArrDelayed" === "NO").
    count

res3: Long = 44985


In [12]:
airlines.
    filter(airlines("IsDepDelayed") === "YES" and airlines("IsArrDelayed") === "NO").
    count

res4: Long = 44985


Obtenir le décompte des vols qui partent en retard de plus de 60 minutes par rapport à l'origine.

In [13]:
airlines.
    filter("DepDelay > 60").
    count

res5: Long = 12693


In [14]:
import org.apache.spark.sql.functions.col

import org.apache.spark.sql.functions.col


In [15]:
airlines.
    filter(col("DepDelay") > 60).
    count

res6: Long = 12693


In [16]:
import spark.implicits._

import spark.implicits._


In [17]:
airlines.
    filter($"DepDelay" > 60).
    count

res7: Long = 12693


In [18]:
airlines.
    filter(airlines("DepDelay") > 60).
    count

res8: Long = 12693


Obtenir le décompte des vols qui partent tôt ou à l'heure mais arrivent en retard d'au moins 15 minutes.

In [19]:
airlines.
    filter("IsDepDelayed = 'NO' AND ArrDelay >= 15").
    count

res9: Long = 17834


In [20]:
import org.apache.spark.sql.functions.col

import org.apache.spark.sql.functions.col


In [21]:
airlines.
    filter(col("IsDepDelayed") === "NO" and col("ArrDelay") >= 15).
    count

res10: Long = 17834


In [22]:
import spark.implicits._

import spark.implicits._


In [23]:
airlines.
    filter($"IsDepDelayed" === "NO" and $"ArrDelay" >= 15).
    count

res11: Long = 17834


In [24]:
airlines.
    filter(airlines("IsDepDelayed") === "NO" and airlines("ArrDelay") >= 15).
    count

res12: Long = 17834


Obtenir le nombre de vols qui quittent les principaux aéroports suivants - ORD, DFW, ATL, LAX, SFO.

In [25]:
airlines_all.
    filter("Origin IN ('ORD', 'DFW', 'ATL', 'LAX', 'SFO')").
    select("Origin").
    distinct.
    show

+------+
|Origin|
+------+
|   DFW|
|   SFO|
|   ATL|
|   ORD|
|   LAX|
+------+



In [26]:
airlines_all.
    filter("Origin IN ('ORD', 'DFW', 'ATL', 'LAX', 'SFO')").
    count

res14: Long = 99082


In [27]:
import org.apache.spark.sql.functions.col

import org.apache.spark.sql.functions.col


In [28]:
airlines_all.
    filter(col("Origin") isin ("ORD", "DFW", "ATL", "LAX", "SFO")).
    count

res15: Long = 99082


In [29]:
import spark.implicits._

import spark.implicits._


In [30]:
airlines_all.
    filter($"Origin" isin ("ORD", "DFW", "ATL", "LAX", "SFO")).
    count

res16: Long = 99082


In [31]:
airlines_all.
    filter(airlines_all("Origin") isin ("ORD", "DFW", "ATL", "LAX", "SFO")).
    count

res17: Long = 99082


Obtenir le nombre de vols partis en retard entre le 1er janvier 2008 et le 9 janvier en utilisant FlightDate.
La date doit être au format yyyyMMdd.

In [32]:
import org.apache.spark.sql.functions.{lpad, concat, col}

import org.apache.spark.sql.functions.{lpad, concat, col}


In [33]:

airlines.
    withColumn("FlightDate", 
               concat(col("Year"), 
                      lpad(col("Month"), 2, "0"), 
                      lpad(col("DayOfMOnth"), 2, "0")
                     )
              ).
    show

+----+-----+----------+--------+--------+-------------+---------+------------+------------+----------+
|Year|Month|DayOfMonth|DepDelay|ArrDelay|UniqueCarrier|FlightNum|IsArrDelayed|IsDepDelayed|FlightDate|
+----+-----+----------+--------+--------+-------------+---------+------------+------------+----------+
|1988|    1|         9|      17|      23|           PI|      942|         YES|         YES|  19880109|
|1988|    1|        29|       8|       7|           PI|      942|         YES|         YES|  19880129|
|1988|    1|        23|       0|      -9|           PI|      943|          NO|          NO|  19880123|
|1988|    1|        18|      14|      28|           PI|      943|         YES|         YES|  19880118|
|1988|    1|        10|       3|      12|           PI|      944|         YES|         YES|  19880110|
|1988|    1|        30|       0|       8|           PI|      944|         YES|          NO|  19880130|
|1988|    1|        20|      45|      50|           PI|      944|        

In [34]:
airlines.
    withColumn("FlightDate", 
               concat(col("Year"), 
                      lpad(col("Month"), 2, "0"), 
                      lpad(col("DayOfMOnth"), 2, "0")
                     )
              ).
    filter("FlightDate LIKE '2008010%' AND IsDepDelayed = 'YES'").
    count

res19: Long = 0


In [35]:
airlines.
    withColumn("FlightDate", 
               concat(col("Year"), 
                      lpad(col("Month"), 2, "0"), 
                      lpad(col("DayOfMOnth"), 2, "0")
                     )
              ).
    filter("FlightDate BETWEEN '20080101' AND '20080109' AND IsDepDelayed = 'YES'").
    count

res20: Long = 0


In [36]:
airlines.
    withColumn("FlightDate", 
               concat(col("Year"), 
                      lpad(col("Month"), 2, "0"), 
                      lpad(col("DayOfMOnth"), 2, "0")
                     )
              ).
    filter($"FlightDate" like "2008010%" and $"IsDepDelayed" === "YES").
    count

res21: Long = 0


In [52]:
airlines.
    withColumn("FlightDate", 
               concat(col("Year"), 
                      lpad(col("Month"), 2, "0"), 
                      lpad(col("DayOfMOnth"), 2, "0")
                     )
              ).
    filter($"FlightDate" between ("20080101", "20080109") and $"IsDepDelayed" === "YES").
    count

res28: Long = 4629


### Transformation et extractions d'informations dans retail_db

**Problème**

Obtenez tous les clients qui ont passé une commande d'un montant supérieur à 200.
Les fichiers d'entrée sont des fichiers délimités par des virgules placés sous l'emplacement datasets/retail_db :

datasets/retail_db/customers  
datasets/retail_db/orders  
datasets/retail_db/order_items  

**Schéma pour les fichiers d'entrée**

```Client
Customer_id,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode
 
Ordres
Order_id,order_date,order_customer_id,order_status
 
Orders_Item
Order_item_id,Order_item_order_id,order_item_product_id,Order_item_quantity,Order_item_subtotal,Order_item_product_price
```

**Sortie :**

La sortie doit être placée sous l'emplacement datasets/retail_db/output

Le fichier de sortie doit être un fichier séparé par des tabulations

La sortie doit avoir customer_fname,customer_lname,customer_city,order_amount.

L'en-tête ci-dessous doit être ajouté à la sortie

fname, lname, city, order_amount

**Lecture des donnees clients**

In [39]:
val customers = spark.
                read.
                schema("""customer_id INT, customer_fname STRING, customer_lname STRING, 
                customer_email STRING, customer_password STRING, customer_street STRING, 
                customer_city STRING, customer_state STRING, customer_zipcode STRING""").
                option("sep", ",").
                csv("datasets/retail_db/customers")

customers: org.apache.spark.sql.DataFrame = [customer_id: int, customer_fname: string ... 7 more fields]


In [40]:
customers.show

+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|
+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|          1|       Richard|     Hernandez|     XXXXXXXXX|        XXXXXXXXX|  6303 Heather Plaza|  Brownsville|            TX|           78521|
|          2|          Mary|       Barrett|     XXXXXXXXX|        XXXXXXXXX|9526 Noble Embers...|    Littleton|            CO|           80126|
|          3|           Ann|         Smith|     XXXXXXXXX|        XXXXXXXXX|3422 Blue Pioneer...|       Caguas|            PR|           00725|
|          4|          Mary|         Jones|     XXXXXXXXX|        XXXXXXXXX|  8324 Little Common|   San Marcos|            CA|          

In [41]:
val orders = spark.
             read.
             schema("""order_id INT, order_date STRING, order_customer_id INT, order_status STRING""").
             option("sep", ",").csv("datasets/retail_db/orders")

orders: org.apache.spark.sql.DataFrame = [order_id: int, order_date: string ... 2 more fields]


In [42]:
orders.show(false)

+--------+---------------------+-----------------+---------------+
|order_id|order_date           |order_customer_id|order_status   |
+--------+---------------------+-----------------+---------------+
|1       |2013-07-25 00:00:00.0|11599            |CLOSED         |
|2       |2013-07-25 00:00:00.0|256              |PENDING_PAYMENT|
|3       |2013-07-25 00:00:00.0|12111            |COMPLETE       |
|4       |2013-07-25 00:00:00.0|8827             |CLOSED         |
|5       |2013-07-25 00:00:00.0|11318            |COMPLETE       |
|6       |2013-07-25 00:00:00.0|7130             |COMPLETE       |
|7       |2013-07-25 00:00:00.0|4530             |COMPLETE       |
|8       |2013-07-25 00:00:00.0|2911             |PROCESSING     |
|9       |2013-07-25 00:00:00.0|5657             |PENDING_PAYMENT|
|10      |2013-07-25 00:00:00.0|5648             |PENDING_PAYMENT|
|11      |2013-07-25 00:00:00.0|918              |PAYMENT_REVIEW |
|12      |2013-07-25 00:00:00.0|1837             |CLOSED      

In [45]:
val order_items = spark.
                  read.
                  schema("""order_item_id INT, order_item_order_id INT, order_item_product_id INT, 
                  order_item_quantity INT, order_item_subtotal DOUBLE, order_item_product_price DOUBLE""").
                  option("sep", ",").csv("datasets/retail_db/order_items")

order_items: org.apache.spark.sql.DataFrame = [order_item_id: int, order_item_order_id: int ... 4 more fields]


In [46]:
order_items.show(false)

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|1            |1                  |957                  |1                  |299.98             |299.98                  |
|2            |2                  |1073                 |1                  |199.99             |199.99                  |
|3            |2                  |502                  |5                  |250.0              |50.0                    |
|4            |2                  |403                  |1                  |129.99             |129.99                  |
|5            |4                  |897                  |2                  |49.98              |24.99                   |
|6            |4

**Jointure** avec les Dataframes

In [74]:
val requete = order_items.
       groupBy($"order_item_order_id").
       sum("order_item_subtotal").
       where(" sum(order_item_subtotal) > 200").
       join(orders, $"order_id" === $"order_item_order_id").
       join(customers, $"order_customer_id" === $"customer_id").
       select($"customer_fname".as("fname"),$"customer_lname".as("lname"),$"customer_city".as("city"), $"sum(order_item_subtotal)".as("order_amount")).
       orderBy($"order_amount".desc)

requete: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [fname: string, lname: string ... 2 more fields]


In [75]:
requete.show

+---------+--------+---------------+------------------+
|    fname|   lname|           city|      order_amount|
+---------+--------+---------------+------------------+
| Victoria|   Smith|         Caguas|3449.9100000000003|
|     Mary|Anderson|        Augusta|2859.8900000000003|
| Samantha|   Smith|       Palatine|           2839.91|
|   Daniel|   Smith|         Caguas|           2779.86|
|     Mary|    Lane|         Caguas|            2699.9|
|  Phillip|   Smith|       Amarillo|           2629.92|
|   Teresa|    Gray|         Caguas|           2629.92|
|Catherine| Hawkins|         Caguas|            2629.9|
|    Amber|Sheppard|         Caguas|           2399.96|
|  Matthew|   Smith|        Seattle|           2399.95|
|     Mary|   Clark|      Frankfort|           2349.89|
|     Mary|Cantrell|         Caguas|           2329.94|
|  Richard|   Perry|          Bronx|           2299.96|
|    Helen|   Smith|         Caguas|           2259.95|
|  Charles|  Rogers|      San Diego|           2

In [49]:
requete.write.mode("overwrite").option("header", "true").option("sep", "\t").format("csv").save("datasets/retail_db/output")

In [68]:
order_items.
       groupBy($"order_item_order_id").
       sum("order_item_subtotal").
       where(" sum(order_item_subtotal) > 200").show

+-------------------+------------------------+
|order_item_order_id|sum(order_item_subtotal)|
+-------------------+------------------------+
|                148|                  479.99|
|                463|       829.9200000000001|
|                496|      441.95000000000005|
|               1088|      249.97000000000003|
|               1580|                  299.95|
|               1591|                  439.86|
|               1645|      1509.7900000000002|
|               2366|                  299.97|
|               2659|       724.9100000000001|
|               2866|                  569.96|
|               3175|                  209.97|
|               3794|                  299.95|
|               3918|       829.9300000000001|
|               3997|                  579.95|
|               4818|                  399.98|
|               5156|                 1199.93|
|               5300|                  759.85|
|               5803|       764.8599999999999|
|            

### Spark SQL 

Nous allons reprendre le meme exercice avec Spark SQL

D'abords on crée des tables temporaires à partir des dataframes avec `.createTempView` ou `.createOrReplaceTempView`

In [51]:
customers.createOrReplaceTempView("customers")

In [52]:
orders.createOrReplaceTempView("orders")

In [53]:
order_items.createOrReplaceTempView("order_items")

In [78]:
val sqlrequete = spark.sql("SELECT customer_fname as fname, customer_lname as lname, customer_city as city, order_amount " +
        "FROM (SELECT order_item_order_id, sum(order_item_subtotal) as order_amount  FROM order_items  GROUP BY " +
        "order_item_order_id HAVING sum(order_item_subtotal) > 200)"+
        "JOIN orders ON order_id = order_item_order_id JOIN customers ON order_customer_id = customer_id " +
        "ORDER BY order_amount DESC ")

sqlrequete: org.apache.spark.sql.DataFrame = [fname: string, lname: string ... 2 more fields]


In [79]:
sqlrequete.show

+---------+--------+---------------+------------------+
|    fname|   lname|           city|      order_amount|
+---------+--------+---------------+------------------+
| Victoria|   Smith|         Caguas|3449.9100000000003|
|     Mary|Anderson|        Augusta|2859.8900000000003|
| Samantha|   Smith|       Palatine|           2839.91|
|   Daniel|   Smith|         Caguas|           2779.86|
|     Mary|    Lane|         Caguas|            2699.9|
|   Teresa|    Gray|         Caguas|           2629.92|
|  Phillip|   Smith|       Amarillo|           2629.92|
|Catherine| Hawkins|         Caguas|            2629.9|
|    Amber|Sheppard|         Caguas|           2399.96|
|  Matthew|   Smith|        Seattle|           2399.95|
|     Mary|   Clark|      Frankfort|           2349.89|
|     Mary|Cantrell|         Caguas|           2329.94|
|  Richard|   Perry|          Bronx|           2299.96|
|    Helen|   Smith|         Caguas|           2259.95|
|  Charles|  Rogers|      San Diego|           2

In [80]:
sqlrequete.write.mode("overwrite").option("header", "true").option("sep", "\t").format("csv").save("datasets/retail_db/output")